In [89]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.naive_bayes import GaussianNB
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
df1= pd.read_csv("Dry_Bean_Dataset.csv")
df2= pd.read_csv("default of credit card clients.csv")
df3= pd.read_csv("car.data")

 Naive Bayes

In [29]:
X = df1.drop('Class', axis=1)
y = df1['Class']
gnb = GaussianNB()
loo = LeaveOneOut()
accuracies = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy with Leave-One-Out Cross-Validation: {average_accuracy}")

Average Accuracy with Leave-One-Out Cross-Validation: 0.7633531702299611


In [11]:
with open("results/naivebayes_leaveoneout_df1.txt", 'w') as file:
    file.write(f"The average accuracy with Leave-One-Out Cross-Validation is: {average_accuracy}\n")

In [30]:
gnb = GaussianNB()
scores = cross_val_score(gnb, X, y, cv=10)
average_accuracy = scores.mean()
print(f"Average Accuracy with 10-Fold Cross-Validation: {average_accuracy}")

Average Accuracy with 10-Fold Cross-Validation: 0.7242245433682799


In [31]:
with open('results/naive_bayes_10_fold_accuracy_df1.txt', 'w') as file:
    file.write(f"The average accuracy with 10-Fold Cross-Validation is: {average_accuracy}\n")

In [16]:
for col in df2.columns:
    if df2[col].dtype == 'object':
        df2[col] = df2[col].fillna(df2[col].mode()[0])
    else:
        df2[col] = df2[col].fillna(0)
le = LabelEncoder()
categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE']
for column in categorical_columns:
    df2[column] = le.fit_transform(df2[column])
X = df2.drop('default payment next month', axis=1)
y = df2['default payment next month']
class NaiveBayesGaussian:
    def __init__(self):
        self.mean = None
        self.std = None
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.class_stats = {}
        for cls in self.classes:
            X_cls = X[y == cls]
            self.class_stats[cls] = {}
            for i in range(n_features):
                self.class_stats[cls][i] = {'mean': np.mean(X_cls[:, i]), 'std': np.std(X_cls[:, i])}
    def predict(self, X):
        n_samples, n_features = X.shape
        y_pred = []
        for idx, x_test_row in enumerate(X):
            log_prob_sum = {}
            for cls in self.classes:
                log_prob_sum[cls] = 0.0
                for i in range(n_features):
                    feature_mean, feature_std = self.class_stats[cls][i]['mean'], self.class_stats[cls][i]['std']
                    log_prob_sum[cls] += norm.logpdf(x_test_row[i], feature_mean, feature_std)
            y_pred.append(max(log_prob_sum, key=log_prob_sum.get))
        return y_pred
def loocv_accuracy(model, X, y):
    n_samples = len(y)
    correct_predictions = 0
    for i in range(n_samples):
        X_train = np.concatenate((X[:i], X[(i+1):]))
        y_train = np.concatenate((y[:i], y[(i+1):]))
        model.fit(X_train, y_train)
        y_pred = model.predict(np.array([X[i]]))
        if y_pred[0] == y[i]:
            correct_predictions += 1
    return correct_predictions / n_samples
nb_model = NaiveBayesGaussian()
accuracy = loocv_accuracy(nb_model, X.values, y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.48


In [17]:
with open("results/naivebayes_leaveoneout_df2.txt", 'w') as file:
    file.write(f"The average accuracy with Leave-One-Out Cross-Validation is: {accuracy}\n")

In [32]:
X = df2.drop(columns=['ID', 'default payment next month'])
y = df2['default payment next month']
gnb = GaussianNB()
scores = cross_val_score(gnb, X, y, cv=10, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

Cross-validation scores: [0.35633333 0.38       0.372      0.37533333 0.37833333 0.35633333
 0.37633333 0.39466667 0.4        0.39033333]
Average cross-validation score: 0.3779666666666667


In [33]:
with open('results/naive_bayes_10_fold_accuracy_df2.txt', 'w') as file:
    file.write(f"Cross-validation scores: {scores}\n Average cross-validation score:{ scores.mean()}")

In [22]:
target_col = df3.columns[-1]
le = LabelEncoder()
for col in df3.columns:
    if df3[col].dtype == 'object':
        df3[col] = le.fit_transform(df3[col])
X = df3.drop(target_col, axis=1).values
y = df3[target_col].values
gnb = GaussianNB()
loo = LeaveOneOut()
accuracy = 0
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gnb.fit(X_train, y_train)
    predicted_label = gnb.predict(X_test)
    if predicted_label[0] == y_test[0]:
        accuracy += 1
print(f"Accuracy: {accuracy / len(y) * 100:.2f}%")

Accuracy: 62.25%


In [23]:
with open("results/naivebayes_leaveoneout_df3.txt", 'w') as file:
    file.write(f"The average accuracy with Leave-One-Out Cross-Validation is: {accuracy}\n")

In [25]:
target_col = df3.columns[-1]
le = LabelEncoder()
for col in df3.columns:
    if df3[col].dtype == 'object':
        df3[col] = le.fit_transform(df3[col])
X = df3.drop(target_col, axis=1).values
y = df3[target_col].values
gnb = GaussianNB()
kf = KFold(n_splits=10)
accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gnb.fit(X_train, y_train)
    predicted_labels = gnb.predict(X_test)
    accuracy = np.sum(predicted_labels == y_test) / len(y_test)
    accuracies.append(accuracy)
overall_accuracy = sum(accuracies) / len(accuracies)
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")

Overall Accuracy: 60.22%


In [26]:
with open('results/naive_bayes_10_fold_accuracy_df3.txt', 'w') as file:
    file.write(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")

knn

In [3]:
X = df1.drop('Class', axis=1)
y = df1['Class']
accuracies = []
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    loo = LeaveOneOut()
    accuracies_k = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accuracies_k.append(accuracy_score(y_test, y_pred))
    accuracies.append((k, sum(accuracies_k) / len(accuracies_k)))
accuracies_df = pd.DataFrame(accuracies, columns=['K', 'Accuracy'])
print(accuracies_df)

    K  Accuracy
0   1  0.753655
1   2  0.720741
2   3  0.744692
3   4  0.738079
4   5  0.736169
5   6  0.733524
6   7  0.731981
7   8  0.728161
8   9  0.727059
9  10  0.723900
The optimal K value is 1 with an accuracy of 0.7537.


In [4]:

optimal_k = accuracies_df.loc[accuracies_df['Accuracy'].idxmax(), 'K']
highest_accuracy = accuracies_df['Accuracy'].max()
print(f"The optimal K value is {optimal_k} with an accuracy of {highest_accuracy:.4f}.")

The optimal K value is 1 with an accuracy of 0.7537.


In [7]:
with open("results/knn_leaveoneout_df1.txt", 'w') as file:
    file.write(f"{accuracies_df}")
    file.write(f"\nThe optimal K value is {optimal_k} with an accuracy of {highest_accuracy:.4f}.")

In [3]:
X = df2.drop(columns=['ID', 'default payment next month'])
y = df2['default payment next month']
loo = LeaveOneOut()
def loo_cv_knn(X, y, k):
    y_true, y_pred = [], []
    knn = KNeighborsClassifier(n_neighbors=k)
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred.append(knn.predict(X_test)[0])
        y_true.append(y_test.values[0])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy
k_values = range(1, 11)
accuracies = []
for k in k_values:
    accuracy = loo_cv_knn(X, y, k)
    accuracies.append(accuracy)
    print(f"Accuracy for k={k}: {accuracy:.4f}")

Accuracy for k=1: 0.6957
Accuracy for k=2: 0.7661
Accuracy for k=3: 0.7377
Accuracy for k=4: 0.7697
Accuracy for k=5: 0.7553
Accuracy for k=6: 0.7718
Accuracy for k=7: 0.7639
Accuracy for k=8: 0.7714
Accuracy for k=9: 0.7678
Accuracy for k=10: 0.7739

Best k: 10
Best accuracy: 0.7739


In [4]:
best_k = k_values[accuracies.index(max(accuracies))]
best_accuracy = max(accuracies)
print(f"\nBest k: {best_k}")
print(f"Best accuracy: {best_accuracy:.4f}")


Best k: 10
Best accuracy: 0.7739


In [6]:
with open("results/knn_leaveoneout_df2.txt", 'w') as file:
    file.write(f"{accuracies}")
    file.write(f"\nBest k: {best_k}\nBest accuracy: {best_accuracy:.4f}")

In [9]:
target_col = df3.columns[-1]
le = LabelEncoder()
for col in df3.columns:
    if df3[col].dtype == 'object':
        df3[col] = le.fit_transform(df3[col])
X = df3.drop(target_col, axis=1).values
y = df3[target_col].values
accuracies = []
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    loo = LeaveOneOut()
    accuracy = 0
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        knn.fit(X_train, y_train)
        predicted_label = knn.predict(X_test)
        if predicted_label[0] == y_test[0]:
            accuracy += 1
    accuracies.append(accuracy / len(y) * 100)
best_k = accuracies.index(max(accuracies)) + 1
highest_accuracy = max(accuracies)
print(f"The highest accuracy was achieved with k={best_k} at {highest_accuracy:.2f}%.")

The highest accuracy was achieved with k=9 at 95.19%.


In [10]:
print(accuracies)

[80.37058482918356, 78.92298784018529, 80.89171974522293, 80.77591198610307, 84.88708743485813, 87.60856977417487, 92.81991893456862, 94.20961204400695, 95.19397799652577, 93.9200926462073]


In [11]:
with open("results/knn_leaveoneout_df3.txt", 'w') as file:
    file.write(f"{accuracies}")
    file.write(f"\nThe highest accuracy was achieved with k={best_k} at {highest_accuracy:.2f}%.")

In [9]:
scaler = StandardScaler()
features = df1.drop('Class', axis=1)
target = df1['Class']
scaled_features = scaler.fit_transform(features)
scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns)
X = scaled_features_df
y = target
def evaluate_knn(X, y):
    accuracies = []
    for k in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
        accuracies.append(scores.mean())
    return accuracies
accuracies = evaluate_knn(X, y)
print("Accuracy for each K value:", accuracies)
optimal_k = accuracies.index(max(accuracies)) + 1
print(f"Optimal K value: {optimal_k}")
print(f"Highest accuracy achieved: {max(accuracies)}")

Accuracy for each K value: [0.8585835650343479, 0.8577034788059656, 0.8798913190072515, 0.8834184072564766, 0.8851072082482323, 0.8855487079229339, 0.8847399392128746, 0.888193552076354, 0.8892948197155714, 0.8911320280393294]
Optimal K value: 10
Highest accuracy achieved: 0.8911320280393294


In [10]:
with open("results/knn_10fold_df1.txt", 'w') as file:
    file.write(f"Accuracy for each K value:{accuracies}")
    file.write(f"\nOptimal K value: {optimal_k}\nHighest accuracy achieved: {max(accuracies)}")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [12]:
df2['default payment next month'] = df2['default payment next month'].astype(int)
X = df2.drop('default payment next month', axis=1)
y = df2['default payment next month']
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracies = {}
for k in range(1, 11):
    accuracies[k] = []
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        predictions = knn.predict(X_val)
        accuracy = accuracy_score(y_val, predictions)
        accuracies[k].append(accuracy)
best_k = max(accuracies, key=lambda k: sum(accuracies[k])/len(accuracies[k]))
highest_accuracy = sum(accuracies[best_k]) / len(accuracies[best_k])
print(f"The best k value is {best_k} with an average accuracy of {highest_accuracy:.4f}.")

The best k value is 10 with an average accuracy of 0.7736.


In [13]:
accuracies

{1: [0.684,
  0.697,
  0.6916666666666667,
  0.6913333333333334,
  0.6913333333333334,
  0.687,
  0.6823333333333333,
  0.6903333333333334,
  0.6836666666666666,
  0.6913333333333334],
 2: [0.765,
  0.765,
  0.7663333333333333,
  0.7643333333333333,
  0.765,
  0.762,
  0.7663333333333333,
  0.7583333333333333,
  0.7683333333333333,
  0.769],
 3: [0.7253333333333334,
  0.738,
  0.7336666666666667,
  0.733,
  0.7223333333333334,
  0.7303333333333333,
  0.7226666666666667,
  0.724,
  0.737,
  0.7336666666666667],
 4: [0.7616666666666667,
  0.771,
  0.766,
  0.7706666666666667,
  0.7683333333333333,
  0.7633333333333333,
  0.768,
  0.7656666666666667,
  0.7736666666666666,
  0.7673333333333333],
 5: [0.742,
  0.762,
  0.7513333333333333,
  0.751,
  0.746,
  0.7453333333333333,
  0.75,
  0.749,
  0.7576666666666667,
  0.7496666666666667],
 6: [0.7686666666666667,
  0.7706666666666667,
  0.7643333333333333,
  0.7726666666666666,
  0.7696666666666667,
  0.771,
  0.767,
  0.7676666666666667,
 

In [14]:
with open("results/knn_10fold_df2.txt", 'w') as file:
    file.write(f"Accuracy for each K value:{accuracies}")
    file.write(f"\nThe best k value is {best_k} with an average accuracy of {highest_accuracy:.4f}.")

In [16]:
X = df3.iloc[:, :-1].values
y = df3.iloc[:, -1].values
le = LabelEncoder()
for col in range(X.shape[1]):
    X[:, col] = le.fit_transform(X[:, col])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
best_k = 1
best_accuracy = 0
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"K: {k}, Accuracy: {accuracy}")
    if accuracy > best_accuracy:
        best_k = k
        best_accuracy = accuracy

K: 1, Accuracy: 0.8362235067437379
K: 2, Accuracy: 0.8208092485549133
K: 3, Accuracy: 0.8747591522157996
K: 4, Accuracy: 0.8670520231213873
K: 5, Accuracy: 0.905587668593449
K: 6, Accuracy: 0.9094412331406551
K: 7, Accuracy: 0.9113680154142582
K: 8, Accuracy: 0.8901734104046243
K: 9, Accuracy: 0.8921001926782274
K: 10, Accuracy: 0.8631984585741811

The best K is 7 with an accuracy of 0.91.


In [17]:
print(f"\nThe best K is {best_k} with an accuracy of {best_accuracy:.2f}.")


The best K is 7 with an accuracy of 0.91.


In [19]:
with open("results/knn_10fold_df3.txt", 'w') as file:
    for k in range(1, 11):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        file.write(f"K: {k}, Accuracy: {accuracy}\n")
        if accuracy > best_accuracy:
            best_k = k
            best_accuracy = accuracy
    file.write(f"\nThe best K is {best_k} with an accuracy of {best_accuracy:.2f}.")

random forest

In [9]:
X = df1.drop(columns=['Class']).head(1000)
y = df1['Class'].head(1000)
loo = LeaveOneOut()
def loo_cv_random_forest(X, y):
    y_true, y_pred = [], []
    rf = RandomForestClassifier(random_state=42)
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        rf.fit(X_train, y_train)
        y_pred.append(rf.predict(X_test)[0])
        y_true.append(y_test.values[0])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy
accuracy = loo_cv_random_forest(X, y)
print(f"Accuracy with Random Forest and Leave-One-Out Cross-Validation: {accuracy:.4f}")

Accuracy with Random Forest and Leave-One-Out Cross-Validation: 1.0000


In [10]:
with open("results/randomforest_leaveoneout_df1.txt", 'w') as file:
    file.write(f"Accuracy with Random Forest and Leave-One-Out Cross-Validation: {accuracy:.4f}\n")

In [11]:
X = df2.drop(columns=['ID', 'default payment next month']).head(100)
y = df2['default payment next month'].head(100)
loo = LeaveOneOut()
def loo_cv_random_forest(X, y):
    y_true, y_pred = [], []
    rf = RandomForestClassifier(random_state=42)
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        rf.fit(X_train, y_train)
        y_pred.append(rf.predict(X_test)[0])
        y_true.append(y_test.values[0])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy
accuracy = loo_cv_random_forest(X, y)
print(f"Accuracy with Random Forest and Leave-One-Out Cross-Validation: {accuracy:.4f}")

Accuracy with Random Forest and Leave-One-Out Cross-Validation: 0.7200


In [12]:
with open("results/randomforest_leaveoneout_df2.txt", 'w') as file:
    file.write(f"Accuracy with Random Forest and Leave-One-Out Cross-Validation: {accuracy:.4f}\n")

In [28]:
numerical_features = ['Feature3']
categorical_features = ['Feature1', 'Feature2', 'Feature4', 'Feature5', 'Feature6']
for feature in categorical_features:
    df3[feature] = df3[feature].astype('category')
accuracies = []
for index, row in df3.iterrows():
    X_train = df3.drop(index).drop(columns='Label')
    y_train = df3.drop(index).loc[:, 'Label']
    X_test = row.drop('Label').values.reshape(1, -1)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    y_pred = [clf.predict(X_test)]
    y_true = [row['Label']]
    accuracy = accuracy_score(y_true, y_pred)
    accuracies.append(accuracy)
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy:.2f}")

C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names,

Average Accuracy: 0.99


C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [29]:
with open("results/randomforest_leaveoneout_df3.txt", 'w') as file:
    file.write(f"Average Accuracy: {average_accuracy:.2f}\n")

In [32]:
label_encoder = LabelEncoder()
df1['Class'] = label_encoder.fit_transform(df1['Class'])
X = df1.drop(columns=['Class'])
y = df1['Class']
rf = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf, X, y, cv=kf)
mean_accuracy = cv_scores.mean()
std_accuracy = cv_scores.std()
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation: {std_accuracy:.4f}")

Mean Accuracy: 0.9239
Standard Deviation: 0.0063


In [35]:
with open("results/randomforest_10fold_df1.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")
    file.write(f"Standard Deviation: {std_accuracy:.4f}")

In [36]:
X = df2.drop(columns=['ID', 'default payment next month'])
y = df2['default payment next month']
rf = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf, X, y, cv=kf)
mean_accuracy = cv_scores.mean()
std_accuracy = cv_scores.std()
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation: {std_accuracy:.4f}")

Mean Accuracy: 0.8170
Standard Deviation: 0.0076


In [37]:
with open("results/randomforest_10fold_df2.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")
    file.write(f"Standard Deviation: {std_accuracy:.4f}")

In [41]:
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df3 = pd.read_csv('car.data', header=None, names=column_names)
le = LabelEncoder()
for column in df3.columns:
    df3[column] = le.fit_transform(df3[column])
X = df3.drop(columns=['class'])
y = df3['class']
rf = RandomForestClassifier(random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf, X, y, cv=kf)
mean_accuracy = cv_scores.mean()
std_accuracy = cv_scores.std()
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Standard Deviation: {std_accuracy:.4f}")

Mean Accuracy: 0.9815
Standard Deviation: 0.0112


In [42]:
with open("results/randomforest_10fold_df3.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")
    file.write(f"Standard Deviation: {std_accuracy:.4f}")

REPTree

In [48]:
le = LabelEncoder()
df1['Class'] = le.fit_transform(df1['Class'])
X = df1.drop(columns=['Class']).head(5000)
y = df1['Class'].head(5000)
clf = DecisionTreeClassifier(random_state=42)
loo = LeaveOneOut()
accuracies = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
mean_accuracy = sum(accuracies) / len(accuracies)
print(f"Mean Accuracy: {mean_accuracy:.4f}")

Mean Accuracy: 0.9554


In [49]:
with open("results/reptree_leaveoneout_df1.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")

In [53]:
X = df2.drop(columns=['ID', 'default payment next month']).head(1000)
y = df2['default payment next month'].head(1000)
clf = DecisionTreeClassifier(random_state=42)
loo = LeaveOneOut()
accuracies = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
mean_accuracy = sum(accuracies) / len(accuracies)
print(f"Mean Accuracy: {mean_accuracy:.4f}")

Mean Accuracy: 0.7330


In [54]:
with open("results/reptree_leaveoneout_df2.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")

In [55]:
label_encoders = {}
for column in df3.columns:
    label_encoders[column] = LabelEncoder()
    df3[column] = label_encoders[column].fit_transform(df3[column])
X = df3.drop(columns=['class'])
y = df3['class']
clf = DecisionTreeClassifier(random_state=42)
loo = LeaveOneOut()
accuracies = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
mean_accuracy = sum(accuracies) / len(accuracies)
print(f"Mean Accuracy: {mean_accuracy:.4f}")

Mean Accuracy: 0.9832


In [56]:
with open("results/reptree_leaveoneout_df3.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")

In [69]:
label_encoder = LabelEncoder()
df1['Class'] = label_encoder.fit_transform(df1['Class'])
X = df1.drop(columns=['Class']).head(6000)
y = df1['Class'].head(6000)
clf = DecisionTreeClassifier(random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')
mean_accuracy = scores.mean()
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"All Fold Accuracies: {scores}")

Mean Accuracy: 0.9543
All Fold Accuracies: [0.96166667 0.94333333 0.95666667 0.96       0.95666667 0.95833333
 0.96166667 0.945      0.94333333 0.95666667]


In [70]:
with open("results/reptree_10fold_df1.txt", 'w') as file:
    file.write(f"Mean Accuracy: {mean_accuracy:.4f}\nAll Fold Accuracies: {scores}")

In [71]:
X = df2.drop('default payment next month', axis=1).head(1000)
y = df2['default payment next month'].head(1000)
clf = DecisionTreeClassifier()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=kf)
for i, score in enumerate(scores, 1):
    print(f"Fold {i}: {score}")
print(f"\nMean Accuracy: {scores.mean()}")
print(f"\nStandard Deviation: {scores.std()}")

Fold 1: 0.82
Fold 2: 0.7
Fold 3: 0.64
Fold 4: 0.77
Fold 5: 0.73
Fold 6: 0.78
Fold 7: 0.68
Fold 8: 0.79
Fold 9: 0.73
Fold 10: 0.61

Mean Accuracy: 0.7250000000000001

Standard Deviation: 0.06438167441127948


In [72]:
with open("results/reptree_10fold_df2.txt", 'w') as file:
    file.write(f"Mean Accuracy: {scores.mean():.4f}\nStandard Deviation: {scores.std()}")

In [73]:
# df3 = pd.get_dummies(df3, columns=["buying", "maint", "lug_boot", "safety"])
X = df3.drop('class', axis=1)
y = df3['class']
clf = DecisionTreeClassifier()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=kf)
for i, score in enumerate(scores, 1):
    print(f"Fold {i}: {score}")
print(f"\nMean Accuracy: {scores.mean()}")
print(f"Standard Deviation: {scores.std()}")

Fold 1: 0.9595375722543352
Fold 2: 0.9884393063583815
Fold 3: 0.9653179190751445
Fold 4: 0.9826589595375722
Fold 5: 0.976878612716763
Fold 6: 0.9826589595375722
Fold 7: 0.9884393063583815
Fold 8: 0.9942196531791907
Fold 9: 0.9825581395348837
Fold 10: 0.9767441860465116

Mean Accuracy: 0.9797452614598736
Standard Deviation: 0.010096016269036593


In [74]:
with open("results/reptree_10fold_df3.txt", 'w') as file:
    file.write(f"Mean Accuracy: {scores.mean():.4f}\nStandard Deviation: {scores.std()}")

k-means

In [77]:
X = df1.drop('Class', axis=1).head(5000)
y_true = df1['Class'].head(5000)
class_map = {cls: idx for idx, cls in enumerate(y_true.unique())}
y_true = y_true.map(class_map)
n_clusters = len(class_map)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)
y_pred = kmeans.labels_
labels_map = {}
for cluster in np.unique(y_pred):
    most_common_class = y_true[y_pred == cluster].mode().values[0]
    labels_map[cluster] = most_common_class
y_pred_mapped = np.array([labels_map[label] for label in y_pred])
accuracy = accuracy_score(y_true, y_pred_mapped)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7514


C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [78]:
with open("results/kmeans_df1.txt", 'w') as file:
    file.write(f"Accuracy: {accuracy}")

In [79]:
X = df2.drop(['ID', 'default payment next month'], axis=1)
y_true = df2['default payment next month']
y_true = y_true.astype(int)
n_clusters = len(df2['default payment next month'].unique())
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)
y_pred = kmeans.labels_
labels_map = {}
for cluster in range(n_clusters):
    most_common_class = y_true[y_pred == cluster].mode().values[0]
    labels_map[cluster] = most_common_class
y_pred_mapped = [labels_map[label] for label in y_pred]
accuracy = accuracy_score(y_true, y_pred_mapped)
print(f"Accuracy: {accuracy}")

C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Accuracy: 0.7788


In [80]:
with open("results/kmeans_df2.txt", 'w') as file:
    file.write(f"Accuracy: {accuracy}")

In [82]:
X = df3.drop('class', axis=1)
y_true = df3['class']
n_clusters = len(df3['class'].unique())
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)
y_pred = kmeans.labels_
labels_map = {}
for cluster in range(n_clusters):
    most_common_class = y_true[y_pred == cluster].mode().values[0]
    labels_map[cluster] = most_common_class
y_pred_mapped = [labels_map[label] for label in y_pred]
accuracy = accuracy_score(y_true, y_pred_mapped)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7002314814814815


C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [83]:
with open("results/kmeans_df3.txt", 'w') as file:
    file.write(f"Accuracy: {accuracy}")

farthest first

In [84]:
X = df1.drop('Class', axis=1).head(5000)
y_true = df1['Class'].head(5000)
n_clusters = len(df1['Class'].unique())
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)
y_pred = kmeans.labels_
labels_map = {}
for cluster in range(n_clusters):
    most_common_class = y_true[y_pred == cluster].mode().values[0]
    labels_map[cluster] = most_common_class
y_pred_mapped = [labels_map[label] for label in y_pred]
accuracy = accuracy_score(y_true, y_pred_mapped)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7456


C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [85]:
with open("results/farthestfirst_df1.txt", 'w') as file:
    file.write(f"Accuracy: {accuracy}")

In [92]:
le = LabelEncoder()
for col in df2.columns[:-1]:
    df2[col] = le.fit_transform(df2[col])
num_clusters = len(np.unique(df2['default payment next month']))
Z = linkage(df2.iloc[:, :-1].values, method='complete')
clusters = fcluster(Z, t=0.5, criterion='distance')
df2['Cluster'] = clusters
cluster_to_class = {}
for i, cluster_label in enumerate(set(clusters)):
    cluster_to_class[i] = max(set(df2[df2['Cluster'] == cluster_label]['default payment next month'].value_counts().index))
df2['Predicted_Class'] = df2.apply(lambda x: cluster_to_class[x.name], axis=1)
conf_mat = confusion_matrix(df2['default payment next month'], df2['Predicted_Class'])
accuracy = accuracy_score(df2['default payment next month'], df2['Predicted_Class'])
print("Number of Clusters:", num_clusters)
print("Confusion Matrix:\n", conf_mat)
print("Accuracy:", accuracy)

Number of Clusters: 2
Confusion Matrix:
 [[18182  5182]
 [ 5182  1454]]
Accuracy: 0.6545333333333333


In [93]:
with open("results/farthestfirst_df2.txt", 'w') as file:
    file.write(f"Number of Clusters:{num_clusters}\nConfusion Matrix:\n{conf_mat}\nAccuracy: {accuracy}")

In [100]:
le = LabelEncoder()
for column in df3.columns:
    df3[column] = le.fit_transform(df3[column])
kmeans = KMeans(n_clusters=len(df3['class'].unique()), random_state=0)
clusters = kmeans.fit_predict(df3[['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']])
df3['cluster'] = clusters
cluster_to_class = {}
for cluster in df3['cluster'].unique():
    cluster_data = df3[df3['cluster'] == cluster]
    most_frequent_class = cluster_data['class'].mode()[0]
    cluster_to_class[cluster] = most_frequent_class
df3['predicted_class'] = df3['cluster'].map(cluster_to_class)
accuracy = accuracy_score(df3['class'], df3['predicted_class'])
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.70


C:\Users\NS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [101]:
with open("results/farthestfirst_df3.txt", 'w') as file:
    file.write(f"Accuracy: {accuracy:.2f}")